This notebook was used to compile all of the available data from the Utah Flux Network stations.  It should only need to be used once, as other notebooks are used to comile the newer data.

# Import Relevant Libraries

## Standard Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import sys
import pathlib
import glob
import matplotlib.pyplot as plt
from urllib.parse import quote
from sqlalchemy import create_engine
import configparser
import re

import statsmodels.api as sm
#import pingouin as pg
import plotly.express as px
import plotly.express as px


## Import Libraries

In [2]:
import matplotlib
import pandas
import pathlib
import sys

sys.path.append("../../../Micromet")
import micromet

%matplotlib inline

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
ch = logging.StreamHandler()
ch.setFormatter(
    logging.Formatter(
        fmt="%(levelname)s [%(asctime)s] %(name)s – %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )
)
logger.addHandler(ch)

credentials needed


# Run Compilation

## List Sites for Examination

## Compile Eddy Data

Search folders, reformat table, and save csv files

In [4]:
site_folders = {'US-UTD':'Dugout_Ranch',
                'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                'US-UTW':'Wellington',
                'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                'US-UTP':'Phrag',
                'US-CdM':'Cedar_mesa',
                'US-UTV':'Desert_View_Myton',
                'US-UTN':'Juab',
                'US-UTG':'Green_River'
                }

comp_edd_df = {}

am = micromet.AmerifluxDataProcessor(config_path='../../data/reformatter_vars.yml',
                                     logger=logger)

for key, value in site_folders.items():

    print(key)
    raw_fold = pathlib.Path('G:/Shared drives/UGS_Flux/Data_Downloads/')
    pths = micromet.fix_all_in_parent(raw_fold)
    raw_data = am.raw_file_compile(raw_fold, value, search_str = "*Flux_AmeriFluxFormat*.dat")
    if raw_data is not None:
        am_data = micromet.Reformatter(config_path="../../data/reformatter_vars.yml",
                                       var_limits_csv= "../../data/extreme_values.csv",
                                       drop_soil=False,
                                       logger=logger,
                                       )
        am_df = am_data.prepare(raw_data)
        comp_edd_df[key] = am_df

        am_df.to_csv(f"../../data/station_data/{key}_HH_{am_df['TIMESTAMP_START'].values[0]:}_{am_df['TIMESTAMP_END'].values[-1]:}.csv")

    


US-UTD

✔ All possible files have been checked.
US-UTB

✔ All possible files have been checked.
US-UTJ

✔ All possible files have been checked.
US-UTW

✔ All possible files have been checked.
US-UTE

✔ All possible files have been checked.
US-UTM

✔ All possible files have been checked.


OSError: [Errno 22] Invalid argument

Compile files from each station into a a single dataframe.

In [ ]:
cdf = pd.concat(comp_edd_df, axis=0)
cdf.index.set_names(['stationid','datetime_start'],inplace=True)
#cdf.rename(columns={'level_0':'stationid'},inplace=True)
#cdf.to_parquet('../station_data/all_data.parquet')
for col in cdf.columns:
    cdf.rename(columns={col:col.lower()},inplace=True)

Save to Parquet

In [ ]:
cdf.to_parquet('../../station_data/all_eddy_data.parquet')

In [ ]:

comp_met_df = {}

am = micromet.AmerifluxDataProcessor()

for key, value in site_folders.items():

    print(key)
    raw_fold = pathlib.Path('G:/Shared drives/UGS_Flux/Data_Downloads/')
    raw_data = am.raw_file_compile(raw_fold, value, search_str = "*Statistics_AmeriFlux*.dat")
    if raw_data is not None:
        am_data = micromet.Reformatter(raw_data,
                                       config_path="../../data/reformatter_vars.yml", 
                                       drop_soil=False,
                                       data_type='met'
                                       )
        am_df = am_data.et_data
        comp_met_df[key] = am_df

        #am_df.to_csv(f"../../station_data/{key}_HH_{am_df['TIMESTAMP_START'].values[0]:}_{am_df['TIMESTAMP_END'].values[-1]:}.csv")

        



In [ ]:
mapping = {
    "T_100cm_N_Avg": "TS_3_9_1",
    "BulkEC_5cm_N_Avg": "EC_3_1_1",
    "BulkEC_10cm_N_Avg": "EC_3_2_1",
    "BulkEC_20cm_N_Avg": "EC_3_3_1",
    "BulkEC_30cm_N_Avg": "EC_3_4_1",
    "BulkEC_40cm_N_Avg": "EC_3_5_1",
    "BulkEC_50cm_N_Avg": "EC_3_6_1",
    "BulkEC_60cm_N_Avg": "EC_3_7_1",
    "BulkEC_75cm_N_Avg": "EC_3_8_1",
    "BulkEC_100cm_N_Avg": "EC_3_9_1",
    "VWC_5cm_S_Avg": "SWC_4_1_1",
    "VWC_10cm_S_Avg": "SWC_4_2_1",
    "VWC_20cm_S_Avg": "SWC_4_3_1",
    "VWC_30cm_S_Avg": "SWC_4_4_1",
    "VWC_40cm_S_Avg": "SWC_4_5_1",
    "VWC_50cm_S_Avg": "SWC_4_6_1",
    "VWC_60cm_S_Avg": "SWC_4_7_1",
    "VWC_75cm_S_Avg": "SWC_4_8_1",
    "VWC_100cm_S_Avg": "SWC_4_9_1",
    "Ka_5cm_S_Avg": "K_4_1_1",
    "Ka_10cm_S_Avg": "K_4_2_1",
    "Ka_20cm_S_Avg": "K_4_3_1",
    "Ka_30cm_S_Avg": "K_4_4_1",
    "Ka_40cm_S_Avg": "K_4_5_1",
    "Ka_50cm_S_Avg": "K_4_6_1",
    "Ka_60cm_S_Avg": "K_4_7_1",
    "Ka_75cm_S_Avg": "K_4_8_1",
    "Ka_100cm_S_Avg": "K_4_9_1",
    "T_5cm_S_Avg": "TS_4_1_1",
    "T_10cm_S_Avg": "TS_4_2_1",
    "T_20cm_S_Avg": "TS_4_3_1",
    "T_30cm_S_Avg": "TS_4_4_1",
    "T_40cm_S_Avg": "TS_4_5_1",
    "T_50cm_S_Avg": "TS_4_6_1",
    "T_60cm_S_Avg": "TS_4_7_1",
    "T_75cm_S_Avg": "TS_4_8_1",
    "T_100cm_S_Avg": "TS_4_9_1",
    "BulkEC_5cm_S_Avg": "EC_4_1_1",
    "BulkEC_10cm_S_Avg": "EC_4_2_1",
    "BulkEC_20cm_S_Avg": "EC_4_3_1",
    "BulkEC_30cm_S_Avg": "EC_4_4_1",
    "BulkEC_40cm_S_Avg": "EC_4_5_1",
    "BulkEC_50cm_S_Avg": "EC_4_6_1",
    "BulkEC_60cm_S_Avg": "EC_4_7_1",
    "BulkEC_75cm_S_Avg": "EC_4_8_1",
    "BulkEC_100cm_S_Avg": "EC_4_9_1",
    "VWC_5cm_Avg": "SWC_3_1_1",
    "VWC_10cm_Avg": "SWC_3_2_1",
    "VWC_20cm_Avg": "SWC_3_3_1",
    "VWC_30cm_Avg": "SWC_3_4_1",
    "VWC_40cm_Avg": "SWC_3_5_1",
    "VWC_50cm_Avg": "SWC_3_6_1",
    "VWC_60cm_Avg": "SWC_3_7_1",
    "VWC_75cm_Avg": "SWC_3_8_1",
    "VWC_100cm_Avg": "SWC_3_9_1",
    "Ka_5cm_Avg": "K_3_1_1",
    "Ka_10cm_Avg": "K_3_2_1",
    "Ka_20cm_Avg": "K_3_3_1",
    "Ka_30cm_Avg": "K_3_4_1",
    "Ka_40cm_Avg": "K_3_5_1",
    "Ka_50cm_Avg": "K_3_6_1",
    "Ka_60cm_Avg": "K_3_7_1",
    "Ka_75cm_Avg": "K_3_8_1",
    "Ka_100cm_Avg": "K_3_9_1",
    "T_5cm_Avg": "TS_3_1_1",
    "T_10cm_Avg": "TS_3_2_1",
    "T_20cm_Avg": "TS_3_3_1",
    "T_30cm_Avg": "TS_3_4_1",
    "T_40cm_Avg": "TS_3_5_1",
    "T_50cm_Avg": "TS_3_6_1",
    "T_60cm_Avg": "TS_3_7_1",
    "T_75cm_Avg": "TS_3_8_1",
    "T_100cm_Avg": "TS_3_9_1",
    "BulkEC_5cm_Avg": "EC_3_1_1",
    "BulkEC_10cm_Avg": "EC_3_2_1",
    "BulkEC_20cm_Avg": "EC_3_3_1",
    "BulkEC_30cm_Avg": "EC_3_4_1",
    "BulkEC_40cm_Avg": "EC_3_5_1",
    "BulkEC_50cm_Avg": "EC_3_6_1",
    "BulkEC_60cm_Avg": "EC_3_7_1",
    "BulkEC_75cm_Avg": "EC_3_8_1",
    "BulkEC_100cm_Avg": "EC_3_9_1",
    "BulkEC_3_1_1": "EC_3_1_1",
    "BulkEC_3_2_1": "EC_3_2_1",
    "BulkEC_3_3_1": "EC_3_3_1",
    "BulkEC_3_4_1": "EC_3_4_1",
    "BulkEC_3_5_1": "EC_3_5_1",
    "BulkEC_3_6_1": "EC_3_6_1",
    "BulkEC_3_7_1": "EC_3_7_1",
    "BulkEC_3_8_1": "EC_3_8_1",
    "BulkEC_3_9_1": "EC_3_9_1",
    "KA_3_1_1": "K_3_1_1",
    "KA_3_2_1": "K_3_2_1",
    "KA_3_3_1": "K_3_3_1",
    "KA_3_4_1": "K_3_4_1",
    "KA_3_5_1": "K_3_5_1",
    "KA_3_6_1": "K_3_6_1",
    "KA_3_7_1": "K_3_7_1",
    "KA_3_8_1": "K_3_8_1",
    "KA_3_9_1": "K_3_9_1",
    "VWC_2_1_1": "SWC_3_1_1",
    "VWC_2_2_1": "SWC_3_2_1",
    "VWC_2_3_1": "SWC_3_3_1",
    "VWC_2_4_1": "SWC_3_4_1",
    "VWC_2_5_1": "SWC_3_5_1",
    "VWC_2_6_1": "SWC_3_6_1",
    "VWC_2_7_1": "SWC_3_7_1",
    "VWC_2_8_1": "SWC_3_8_1",
    "VWC_2_9_1": "SWC_3_9_1",
    "VWC_3_1_1": "SWC_4_1_1",
    "VWC_3_2_1": "SWC_4_2_1",
    "VWC_3_3_1": "SWC_4_3_1",
    "VWC_3_4_1": "SWC_4_4_1",
    "VWC_3_5_1": "SWC_4_5_1",
    "VWC_3_6_1": "SWC_4_6_1",
    "VWC_3_7_1": "SWC_4_7_1",
    "VWC_3_8_1": "SWC_4_8_1",
    "VWC_3_9_1": "SWC_4_9_1",
    "T__1_8_1": "TS_3_8_1",
    "KA_4_1_1": "K_4_1_1",
    "KA_4_2_1": "K_4_2_1",
    "KA_4_3_1": "K_4_3_1",
    "KA_4_4_1": "K_4_4_1",
    "KA_4_5_1": "K_4_5_1",
    "KA_4_6_1": "K_4_6_1",
    "KA_4_7_1": "K_4_7_1",
    "KA_4_8_1": "K_4_8_1",
    "KA_4_9_1": "K_4_9_1",
    "BulkEC_4_1_1": "EC_4_1_1",
    "BulkEC_4_2_1": "EC_4_2_1",
    "BulkEC_4_3_1": "EC_4_3_1",
    "BulkEC_4_4_1": "EC_4_4_1",
    "BulkEC_4_5_1": "EC_4_5_1",
    "BulkEC_4_6_1": "EC_4_6_1",
    "BulkEC_4_7_1": "EC_4_7_1",
    "BulkEC_4_8_1": "EC_4_8_1",
    "BulkEC_4_9_1": "EC_4_9_1",
    "KA_2_1_1": "K_4_1_1",
    "KA_2_2_1": "K_4_2_1",
    "KA_2_3_1": "K_4_3_1",
    "KA_2_4_1": "K_4_4_1",
    "KA_2_5_1": "K_4_5_1",
    "KA_2_6_1": "K_4_6_1",
    "KA_2_7_1": "K_4_7_1",
    "KA_2_8_1": "K_4_8_1",
    "KA_2_9_1": "K_4_9_1",
    "BulkEC_2_1_1": "EC_4_1_1",
    "BulkEC_2_2_1": "EC_4_2_1",
    "BulkEC_2_3_1": "EC_4_3_1",
    "BulkEC_2_4_1": "EC_4_4_1",
    "BulkEC_2_5_1": "EC_4_5_1",
    "BulkEC_2_6_1": "EC_4_6_1",
    "BulkEC_2_7_1": "EC_4_7_1",
    "BulkEC_2_8_1": "EC_4_8_1",
    "BulkEC_2_9_1": "EC_4_9_1",
    "T_2_1_1": "TS_4_1_1",
    "T_2_2_1": "TS_4_2_1",
    "T_2_3_1": "TS_4_3_1",
    "T_2_4_1": "TS_4_4_1",
    "T_2_5_1": "TS_4_5_1",
    "T_2_6_1": "TS_4_6_1",
    "T_2_7_1": "TS_4_7_1",
    "T_2_8_1": "TS_4_8_1",
    "T_2_9_1": "TS_4_9_1",
}


In [ ]:
ddf.columns = ddf.columns.str.lower()

for old_col, new_col in mapping.items():
    if str(old_col).lower() in ddf.columns.str.lower():
        if str(new_col).lower() in ddf.columns.str.lower():
            ddf[new_col.lower()] = ddf[[old_col.lower(), new_col.lower()]].max(axis=1)
            ddf = ddf.drop(old_col.lower(), axis=1)
        else:
            ddf = ddf.rename(columns={old_col.lower(): new_col.lower()})



In [ ]:
soildfs

for old_col, new_col in mapping.items():
    if str(old_col).lower() in soildfs.columns.str.lower():
        if str(new_col).lower() in soildfs.columns.str.lower():
            soildfs[new_col.lower()] = soildfs[[old_col.lower(), new_col.lower()]].max(axis=1)
            soildfs = soildfs.drop(old_col.lower(), axis=1)
        else:
            soildfs = soildfs.rename(columns={old_col.lower(): new_col.lower()})
    elif str(old_col).lower()+"_eddy" in soildfs.columns.str.lower():
        print(f"Found {old_col} eddy column")
        if str(new_col).lower()+"_eddy" in soildfs.columns.str.lower():
            soildfs[new_col.lower()] = soildfs[[old_col.lower()+"_eddy", new_col.lower()+"_eddy"]].max(axis=1)
            soildfs = soildfs.drop(old_col.lower()+"_eddy", axis=1)
        else:
            soildfs = soildfs.rename(columns={old_col.lower()+"_eddy": new_col.lower()})
    elif str(new_col).lower()+"_eddy" in soildfs.columns.str.lower():
        if str(new_col).lower() in soildfs.columns.str.lower():
            soildfs[new_col.lower()] = soildfs[[new_col.lower()+"_eddy", new_col.lower()+"_eddy"]].max(axis=1)
            soildfs = soildfs.drop(new_col.lower()+"_eddy", axis=1)
            print(f"Found {new_col} eddy column")
        else:
            print(f"Found {new_col} eddy column")
            soildfs = soildfs.rename(columns={new_col.lower()+"_eddy": new_col.lower()})
        


In [ ]:
for old_col, new_col in mapping.items():
    if str(old_col).lower()+"_eddy" in soildfs.columns.str.lower().str.strip():
        print(f"Found {old_col} eddy column")

In [ ]:
"swc_4_1_1"

In [ ]:
soildfs.iloc[0:1,:].to_clipboard()


In [ ]:
ddf = ddf.replace(np.nan, 0)  

In [ ]:
ddf.to_parquet('../../station_data/all_met_data.parquet')

In [ ]:
ddf = pd.concat(comp_met_df, axis=0)
ddf.index.set_names(['stationid','datetime_start'],inplace=True)
#cdf.rename(columns={'level_0':'stationid'},inplace=True)
#cdf.to_parquet('../station_data/all_data.parquet')
for col in ddf.columns:
    ddf.rename(columns={col:col.lower()},inplace=True)

In [ ]:
ddf[~ddf['vwc_2_7_1'].isna()]

In [ ]:
ddf.iloc[0:1,:].to_clipboard()

In [ ]:
import re

soilcols = [col.lower() for col in am_data.MATH_SOILS_V2]
pattern = re.compile(r"2_1_1|1_2_1|1_1_2")
# Print matching columns
matching_cols = [col for col in soilcols if pattern.search(col)]
# Remove them from the original list
soilcols = [col for col in soilcols if not pattern.search(col)]

        
soildfs = pd.merge(ddf,cdf[soilcols],how='left',on=['stationid','datetime_start'],suffixes=(None,'_eddy'))
soildfs

for col in cdf.columns:
    if col in soilcols:
        cdf.drop(columns=col,inplace=True)  # drop the soil columns from the main dataframe

cdf.to_parquet('../../station_data/all_eddy_data.parquet')

soildfs.to_parquet('../../station_data/all_soil_data.parquet')

ddf.to_parquet('../../station_data/all_met_data.parquet')

In [ ]:
cdf = pd.read_parquet('../../station_data/all_eddy_data.parquet')


In [ ]:
cdf.columns

In [ ]:
soildfs = pd.read_parquet('../../station_data/all_soil_data.parquet')
utd_soilt = soildfs.loc['US-UTD'][['ts_3_1_1','ts_3_2_1','ts_3_3_1']].replace(-9999,np.nan)
utd_soilt = utd_soilt[utd_soilt.index >= '2024-07-01']#.resample('30T').mean()
utd_soilt['ts_3_1_1'].plot()
utd_soilt['ts_3_2_1'].shift(-1).plot()
utd_soilt['ts_3_3_1'].shift(-5).plot()
plt.axvline('2024-07-04 15:00',color='r')
#plt.xlim('2024-07-01','2024-07-08')
#plt.ylim(10,35)
plt.grid(True, which='minor')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.signal import correlate

# Function to decompose the seasonal component
def extract_seasonal(ts, period):
    decomposition = seasonal_decompose(ts, model='additive', period=period)
    return decomposition.seasonal

# Function to calculate lag between two seasonal series using cross-correlation
def calculate_lag(seasonal1, seasonal2):
    n = len(seasonal1)
    correlation = correlate(seasonal1 - np.mean(seasonal1), seasonal2 - np.mean(seasonal2), mode='full')
    lags = np.arange(-n + 1, n)
    lag = lags[np.argmax(correlation)]
    return lag, correlation, lags

ts1 = utd_soilt['ts_3_2_1']
ts2 = utd_soilt['ts_3_3_1']
#utd_soilt['ts_3_3_1'].shift(-5).plot()


# Extract seasonal components
seasonal1 = extract_seasonal(ts1, period=48)
seasonal2 = extract_seasonal(ts2, period=48)

# Calculate lag
lag, correlation, lags = calculate_lag(seasonal1.dropna(), seasonal2.dropna())

# Output
print(f"Calculated lag: {lag/2} hours")

# Plot seasonal components and correlation
fig, ax = plt.subplots(3, 1, figsize=(10, 8))

seasonal1.plot(ax=ax[0], label='Seasonal Component 1')
seasonal2.plot(ax=ax[0], label='Seasonal Component 2')
ax[0].legend()
ax[0].set_title('Seasonal Components')
ax[0].set_xlim(pd.to_datetime('2024-07-01'),pd.to_datetime('2024-07-08'))
ax[0].grid(True)

ax[1].plot(lags, correlation)
ax[1].set_title('Cross-Correlation')
ax[1].set_xlabel('Lag (hours)')
ax[1].set_ylabel('Correlation')
ax[1].set_xlim(-10, 10)
ax[1].grid(True)

ax[2].plot(seasonal1.index, seasonal1, label='Series 1')
ax[2].plot(seasonal2.index + pd.Timedelta(hours=lag/2), seasonal2, label='Series 2 (Shifted)')
ax[2].legend()
ax[2].set_title(f'Series alignment (Lag: {lag/2} hours)')
ax[2].set_xlim(pd.to_datetime('2024-07-01'),pd.to_datetime('2024-07-08'))
ax[2].grid(True)
plt.tight_layout()
plt.show()



In [ ]:
cdf = pd.read_parquet('../../station_data/all_eddy_data.parquet')
ddf = pd.read_parquet('../../station_data/all_met_data.parquet')

for col in cdf.columns:
    if col in ddf.columns:
        print(col)


In [ ]:
ddf.head(10).to_clipboard()

In [ ]:
series = ddf.loc['US-UTD','t_si111_body'].replace(-9999,np.nan)
series.plot()
series.diff().plot()
new_series = series[series.diff()<2].diff().cumsum()
new_series.plot()

In [ ]:
config = configparser.ConfigParser()

config.read('../../secrets/config.ini')

from sqlalchemy import create_engine
import urllib.parse
host = config['DEFAULT']['ip']
pw = config['DEFAULT']['pw']
user = config['DEFAULT']['login']

encoded_password = urllib.parse.quote_plus(pw)

def postconn_et(encoded_password, host='localhost',user='postgres',port='5432',db='groundwater', schema = 'groundwater'):
    connection_text = "postgresql+psycopg2://{:}:{:}@{:}:{:}/{:}?gssencmode=disable".format(user,encoded_password,host,port,db)
    return create_engine(connection_text, connect_args={'options': '-csearch_path={}'.format(schema)})


engine = postconn_et(encoded_password, host=host, user=user)

In [ ]:
cdf.to_sql(name = 'amfluxeddy',
           schema='groundwater',
           con=engine,
           if_exists='replace',
           chunksize=2000)

In [ ]:
for col in soildfs.columns:
    print(f"amfluxmet.{col},")

In [ ]:
soildfs.to_sql(name = 'amfluxmet',
           schema='groundwater',
           con=engine,
           if_exists='replace',
           chunksize=2000)